In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sns
import os
from glob import glob


In [ ]:
# read the existing table: 

table = pd.read_csv('/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/11_zonalstats_new/table_catchemnts_from_qgis.csv', index_col = 0)


In [ ]:
table = table.drop('geometry', axis = 1)

In [ ]:
# table.drop_duplicates()

### morphometric parameters from tjalling's book chapter 



In [ ]:
# ruggedness ratio
# Rh = Rd * H (H = range elevation) - requires length of the drainage 


In [ ]:
# melton index 
# M = H *A**0.5 (H = range elevation)

table['M'] = table.range_elevation * (table.area_m ** 0.5)

In [ ]:
# circularity ratio
# Rc = 4piA/P^2
# numpy.pi#


table['Rc'] = (4 * np.pi * table.area_m ) / (table.perimeter ** 2)

In [ ]:
# compactness coefficient  
# Cc = 0.2841* (P/A**0.5)

table['Cc'] = 0.2841 * (table.perimeter / (table.area_m**0.5))



In [ ]:
# delete geometry

# table = table.drop('geometry', axis =1)

In [ ]:
# table.to_csv('morphometrical_params.csv')

## correlations and some related plots

In [ ]:
table_sub = table[['Name', 'x_centroid', 'y_centroid', 'area_m', 'perimeter',
       'glaciersum', 'mean_elevation', 'std_elevation',
       'min_elevation', 'max_elevation', 'range_elevation',
       'variance_elevation','region', 'mean_slope',
       'median_slope', 'std_slope', 'min_slope', 'max_slope', 'range_slope',
       'variance_slope', 'glarea_percent', 'glacier', 'Rc',
       'Cc', 'M']]

table_sub_corr = table_sub.corr()


In [ ]:
# catchments and streams 

plt.figure(figsize=(10, 10))

# mask for the upper triangle
mask = np.triu(np.ones_like(table_sub_corr, dtype=bool))
# color palette
cmap = sns.diverging_palette(230, 20, as_cmap=True)
# heatmap 
sns.heatmap(table_sub_corr, cmap=cmap, mask = mask)
# sns.heatmap(catchments_corr, cmap=cmap)
plt.title('various parameters', fontsize = 15)

# plt.savefig('various_params_corr.png', dpi = 300, bbox_inches='tight')

### how about grouped by 'name' or 'region' plots? 

In [ ]:
# table_sub2 = table[['Name', 'area_m', 'perimeter',
       # 'glaciersum', 'mean_elevation', 'std_elevation',
       # 'min_elevation', 'max_elevation', 'range_elevation',
       # 'variance_elevation', 'mean_slope',
       # 'median_slope', 'std_slope', 'min_slope', 'max_slope', 'range_slope',
       # 'variance_slope', 'glarea_percent', 'glacier', 'Rc',
       # 'Cc', 'M']]
# 
# g = sns.PairGrid(table_sub2, hue = 'Name')
# 
# g.map_diag(sns.histplot)
# g.map_offdiag(sns.scatterplot)
# 
# 
# plt.savefig('params_scatter.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
## test with fewer params 

# table_sub2 = table[['Name', 'area_m', 'perimeter',
#        'glarea_percent', 'mean_elevation', 'M', 'Rc',
#        'Cc', 'mean_slope', 'range_elevation']]

# g = sns.PairGrid(table_sub2, hue = 'Name')
# g.map_diag(sns.histplot)
# g.map_offdiag(sns.scatterplot)

# plt.savefig('params_scatter_sub.png', dpi = 300, bbox_inches = 'tight')



In [ ]:
# table_sub2 = table[['Name', 'area_m', 'perimeter',
#        'glarea_percent', 'mean_elevation', 'M', 'Rc',
#        'Cc', 'mean_slope', 'range_elevation', 'region']]

# g = sns.PairGrid(table_sub2, hue = 'region')
# g.map_diag(sns.histplot)
# g.map_offdiag(sns.scatterplot)



In [ ]:
# make loop to save histograms for each parameter dor different 'name' (ff/df) 

In [ ]:
table.head()

# climate

In [ ]:
## read all temperatures

df = pd.read_csv('/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/10_datawrangl_formodel!!/morphometrical_params.csv', sep = ';', index_col = 0)
df = df[df.target.isin([1,0])]

In [ ]:
len(df.drop_duplicates())

### mean annual temperature 

In [ ]:
# ## -------- mean of all - for one file: 

# # read data 
# climatetemp = xr.open_mfdataset('/Users/varyabazilova/Desktop/era5land/points_coordinates_daymean/*.nc')
# # climatetemp = climatetemp.drop(labels='time_bnds')

# # lats = table.y_wgs
# # lons = table.x_wgs
# # tempsel = climatemp.sel(

# # mean across time:
# climatetemp_mean = climatetemp.mean(dim = 'time')
# # to df
# mean_df = climatetemp_mean.to_dataframe().drop_duplicates().reset_index()
# # rename things
# mean_df1 = mean_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})
# # merge with the rest of the data on the coordinates
# dfclim = df.merge(mean_df1, on=['x_wgs', 'y_wgs'], how='left')#.drop_duplicates()



In [ ]:
%%time
# calculate mean annual temperature from daily data for each year: 

path = '/Users/varyabazilova/Desktop/era5land/daymean_temp'

meanannual = xr.Dataset()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # group by -> resample to annual 
    dsmean = ds.groupby('time.year').mean('time')
    # print(dsmean)
    # merge 
    meanannual = xr.merge([meanannual, dsmean])

# print(meanannual)

meanannual_mean = meanannual.mean(dim='year')

In [ ]:
# select coordinates first 

lats = df.y_wgs
lons = df.x_wgs
meanannual_mean_sel = meanannual_mean.sel(latitude = lats, longitude = lons, method = 'nearest')

# overwrite the coordinates for them to match: (ovewrite dimentions)
meanannual_mean_sel['longitude'] = lons.values # x_wgs
meanannual_mean_sel['latitude']= lats.values   # y_wgs

In [ ]:
# meanannual_mean_sel.longitude.values - df.x_wgs.values


In [ ]:
%%time
# append these values to the table: 
meanannual_mean_sel_df = meanannual_mean_sel.to_dataframe()

len(meanannual_mean_sel_df) **(0.5) 

In [ ]:
meanannual_mean_sel_df = meanannual_mean_sel_df.reset_index()


In [ ]:
len(meanannual_mean_sel_df) **(0.5) 

meanannual_mean_sel_df.head()

# df

In [ ]:
# rename things
meanannual_mean_sel_df1 = meanannual_mean_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})

### dfclim - new table with data

In [ ]:
# merge with the rest of the data on the coordinates
dfclim = df.merge(meanannual_mean_sel_df1, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()

# convert from K to C 
dfclim['t2m'] = dfclim.t2m -273.15

dfclim = dfclim.rename(columns = {'t2m':'mean_annual_T'}) 

### mean jan and june temp (coldest month and warmenst month)

In [ ]:
%%time
# calculate mean annual temperature from daily data for each year: 

path = '/Users/varyabazilova/Desktop/era5land/daymean_temp'

meanmonth = xr.Dataset()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # group by -> resample to annual 
    # dsmean = ds.groupby('time.month').mean('time')
    dsmean = ds.resample(time="1MS").mean(dim="time")
    # print(dsmean)
    # merge 
    meanmonth = xr.merge([meanmonth, dsmean])

# print(meanmonth)

# meanannual_mean = meanannual.mean(dim='year')

In [ ]:
meanmonth = meanmonth.groupby('time.month').mean()
meanmonth_jan = meanmonth.sel(month = 1)
meanmonth_jul = meanmonth.sel(month = 7)

In [ ]:
## select coordinates 

# select coordinates first 

lats = df.y_wgs
lons = df.x_wgs

meanmonth_jan_sel = meanmonth_jan.sel(latitude = lats, longitude = lons, method = 'nearest')
meanmonth_jul_sel = meanmonth_jul.sel(latitude = lats, longitude = lons, method = 'nearest')


# overwrite the coordinates for them to match: (ovewrite dimentions)
meanmonth_jan_sel['longitude'] = lons.values # x_wgs
meanmonth_jan_sel['latitude']= lats.values   # y_wgs

meanmonth_jul_sel['longitude'] = lons.values # x_wgs
meanmonth_jul_sel['latitude']= lats.values   # y_wgs




In [ ]:
%%time
## ---- jan -----
# to data frame
meanmonth_jan_sel_df = meanmonth_jan_sel.to_dataframe().reset_index()
#rename:
meanmonth_jan_sel_df = meanmonth_jan_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})
# convert from K to C 
meanmonth_jan_sel_df['t2m'] = meanmonth_jan_sel_df.t2m -273.15
#rename column: 
meanmonth_jan_sel_df = meanmonth_jan_sel_df.rename(columns = {'t2m':'mean_jan_T'}) 
# drop month 
meanmonth_jan_sel_df = meanmonth_jan_sel_df.drop('month', axis = 1)


## ---- july -----
meanmonth_jul_sel_df = meanmonth_jul_sel.to_dataframe().reset_index()
meanmonth_jul_sel_df = meanmonth_jul_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})
meanmonth_jul_sel_df['t2m'] = meanmonth_jul_sel_df.t2m -273.15
meanmonth_jul_sel_df = meanmonth_jul_sel_df.rename(columns = {'t2m':'mean_july_T'}) 
meanmonth_jul_sel_df = meanmonth_jul_sel_df.drop('month', axis = 1)





In [ ]:
# meanmonth_jan_sel_df
# meanmonth_jul_sel_df

In [ ]:
## merge to the dataframe with other stuff

# january
dfclim = dfclim.merge(meanmonth_jan_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()
# july
dfclim = dfclim.merge(meanmonth_jul_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()



In [ ]:
# dfclim

## mean temp during monsoon and no-monsoon (rest of the year)

### (just choose different months here i guess)

In [ ]:
# monsoon - april to september
# so i need to select only these month
# (1) take all daily values -> mean 
# (2) take mean monthly -? 

# data.where(data.time.dt.month.isin([4,5,6,7,8,9]), drop=True)

In [ ]:
%%time
# calculate mean annual temperature from daily data for each year: 

path = '/Users/varyabazilova/Desktop/era5land/daymean_temp'

dsmonsoon = xr.Dataset()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # select monsoon month:
    dsmons = ds.sel(time=ds.time.dt.month.isin([4,5,6,7,8,9]))
    print(dsmons)
    # merge 
    dsmonsoon = xr.merge([dsmonsoon, dsmons])
print('alltogether', dsmonsoon)

# meanannual_mean = meanannual.mean(dim='year')

In [ ]:
%%time
dsmonsoon_mean = dsmonsoon.mean(dim = 'time')

In [ ]:
dsmonsoon_mean

In [ ]:
# select coordinates: 

# select coordinates first 

lats = df.y_wgs
lons = df.x_wgs

dsmonsoon_mean_sel = dsmonsoon_mean.sel(latitude = lats, longitude = lons, method = 'nearest')


# overwrite the coordinates for them to match: (ovewrite dimentions)
dsmonsoon_mean_sel['longitude'] = lons.values # x_wgs
dsmonsoon_mean_sel['latitude']= lats.values   # y_wgs


In [ ]:
%%time

# to data frame
dsmonsoon_mean_sel_df = dsmonsoon_mean_sel.to_dataframe().reset_index()
#rename:
dsmonsoon_mean_sel_df = dsmonsoon_mean_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})
# convert from K to C 
dsmonsoon_mean_sel_df['t2m'] = dsmonsoon_mean_sel_df.t2m -273.15
#rename column: 
dsmonsoon_mean_sel_df = dsmonsoon_mean_sel_df.rename(columns = {'t2m':'mean_monsoon_T'}) 
# drop month 
# meanmonth_jan_sel_df = meanmonth_jan_sel_df.drop('month', axis = 1)




In [ ]:
dsmonsoon_mean_sel_df.head()

In [ ]:
# merge to the dfclim table on the coordinates:

# july
dfclim = dfclim.merge(dsmonsoon_mean_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()




In [ ]:
# dfclim.to_csv('morph_temperature_monsoon.csv')

### mean annual temp outside of monsoon

In [ ]:
%%time
# calculate mean annual temperature from daily data for each year: 

path = '/Users/varyabazilova/Desktop/era5land/daymean_temp'

dsnomonsoon = xr.Dataset()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # select monsoon month:
    dsnomons = ds.sel(time=ds.time.dt.month.isin([1,2,3,10,11,12]))
    print(dsnomons)
    # merge 
    dsnomonsoon = xr.merge([dsnomonsoon, dsnomons])
print('alltogether', dsnomonsoon)

# meanannual_mean = meanannual.mean(dim='year')

In [ ]:
%%time
dsnomonsoon_mean = dsnomonsoon.mean(dim = 'time')

In [ ]:
%%time
# select coordinates: 

# select coordinates first 

lats = df.y_wgs
lons = df.x_wgs

dsnomonsoon_mean_sel = dsnomonsoon_mean.sel(latitude = lats, longitude = lons, method = 'nearest')


# overwrite the coordinates for them to match: (ovewrite dimentions)
dsnomonsoon_mean_sel['longitude'] = lons.values # x_wgs
dsnomonsoon_mean_sel['latitude']= lats.values   # y_wgs



In [ ]:
%%time

# to data frame
dsnomonsoon_mean_sel_df = dsnomonsoon_mean_sel.to_dataframe().reset_index()
#rename:
dsnomonsoon_mean_sel_df = dsnomonsoon_mean_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})
# convert from K to C 
dsnomonsoon_mean_sel_df['t2m'] = dsnomonsoon_mean_sel_df.t2m -273.15
#rename column: 
dsnomonsoon_mean_sel_df = dsnomonsoon_mean_sel_df.rename(columns = {'t2m':'mean_no_monsoon_T'}) 
# drop month 
# meanmonth_jan_sel_df = meanmonth_jan_sel_df.drop('month', axis = 1)





In [ ]:
dsnomonsoon_mean_sel_df.head()

In [ ]:
# merge to the dfclim table on the coordinates:

dfclim = dfclim.merge(dsnomonsoon_mean_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()



In [ ]:
# dfclim.to_csv('morph_temperature_NOmonsoon.csv')
# dfclim


In [ ]:
# read new file

dfclim = pd.read_csv('/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/10_datawrangl_formodel!!/morph_temperature_NOmonsoon.csv', sep = ',', index_col = 0)
dfclim = dfclim[dfclim.target.isin([1,0])]

### crossing 0 count

In [ ]:
# do for one file: 

file = xr.open_dataset('/Users/varyabazilova/Desktop/era5land/daymean_temp/daymean_temp_1990.nc')
# create new time dim - day of year
# file['time'] = np.arange(1, 366, 1)
# K to C
file['t2m'] = file.t2m -273.15

# shift t2m on 1 timestep back 
file['t2m_shift'] = file.t2m.shift(time = 1)
# create new var with multiplying t by t-1
file['mult'] = file.t2m * file.t2m_shift
# count negative numbers - one d data with only this number 
file['forcount'] = xr.where(file.mult < 0, 1, 0)
# sum to get the number of 0 crossing
file = file.sum(dim = 'time')

In [ ]:
%%time
# calculate mean annual temperature from daily data for each year: 

path = '/Users/varyabazilova/Desktop/era5land/daymean_temp'

meanannual = xr.Dataset()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # group by -> resample to annual 
    dsmean = ds.groupby('time.year').mean('time')
    # print(dsmean)
    # merge 
    meanannual = xr.merge([meanannual, dsmean])

# print(meanannual)

meanannual_mean = meanannual.mean(dim='year')

In [ ]:
%%time
# calculate how many times temp timeseries crosses 0 
path = '/Users/varyabazilova/Desktop/era5land/daymean_temp'

countzeros = xr.Dataset()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # create new time dim - day of year
    # ds['time'] = np.arange(0, 364, 1)
    # K to C 
    ds['t2m'] = ds.t2m -273.15
    # shift t2m on 1 timestep back 
    ds['t2m_shift'] = ds.t2m.shift(time = 1)
    ds = ds.fillna(0)
    # create new var with multiplying t by t-1
    ds['mult'] = ds.t2m * ds.t2m_shift
    # if negative = 1, else = 0 
    ds['forcount'] = xr.where(ds.mult < 0, 1, 0)
    # sum to get the number of 0 crossing
    # countsum = ds.sum(dim = 'time')
    dscount = ds.groupby('time.year').sum('time')
    
    print('countsum', dscount)
    # merge:
    countzeros = xr.merge([countzeros, dscount])

print('count of 0 crossing per year', countzeros)


In [ ]:
countzerosmean = countzeros.mean(dim = 'year')
# countzeros

In [ ]:
# select values 
lats = dfclim.y_wgs
lons = dfclim.x_wgs

countzerosmean_sel = countzerosmean.sel(latitude = lats, longitude = lons, method = 'nearest')

# overwrite the coordinates for them to match: (ovewrite dimentions)
countzerosmean_sel['longitude'] = lons.values # x_wgs
countzerosmean_sel['latitude']= lats.values   # y_wgs

In [ ]:
%%time

# to data frame
countzerosmean_sel_df = countzerosmean_sel.to_dataframe().reset_index()
#rename:
countzerosmean_sel_df = countzerosmean_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})

#rename column: 
countzerosmean_sel_df = countzerosmean_sel_df.rename(columns = {'forcount':'count_zeros'}) 
# drop month 
countzerosmean_sel_df = countzerosmean_sel_df.drop(['t2m', 't2m_shift', 'mult'], axis = 1)

In [ ]:
# countzerosmean_sel_df

In [ ]:
# merge to the dfclim table on the coordinates:

dfclim = dfclim.merge(countzerosmean_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()


In [ ]:
# dfclim.to_csv('morph_temperature_NOmonsoon_zerocount.csv')

## fraction of the year when daily temp is <0

In [ ]:
file = xr.open_dataset('/Users/varyabazilova/Desktop/era5land/daymean_temp/daymean_temp_1990.nc')
file = file.drop(labels='time_bnds')
file['t2m'] = file.t2m -273.15

file['forcount'] = xr.where(file.t2m < 0, 1, 0)

fileyear = file.groupby('time.year').sum('time')
# fraction of the year
fileyear['fraction'] = fileyear.forcount / 365



In [ ]:
# fileyear.fraction.values

In [ ]:
%%time
# calculate how many times temp timeseries crosses 0 
path = '/Users/varyabazilova/Desktop/era5land/daymean_temp'

countnegative = xr.Dataset()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # K to C 
    ds['t2m'] = ds.t2m - 273.15
    # if negative = 1, else = 0 
    ds['forcount'] = xr.where(ds.t2m < 0, 1, 0)
    # sum the negative days number 
    dscount = ds.groupby('time.year').sum('time')
    # fraction of the year
    dscount['fraction'] = dscount.forcount / 365
    
    print('dsfraction', dscount)
    # merge:
    countnegative = xr.merge([countnegative, dscount])

print('negative temp fraction', countnegative)

In [ ]:
countnegative_mean = countnegative.mean(dim = 'year')

In [ ]:
%%time
# select coordinates: 

# select coordinates first 

lats = dfclim.y_wgs
lons = dfclim.x_wgs

countnegative_mean_sel = countnegative_mean.sel(latitude = lats, longitude = lons, method = 'nearest')


# overwrite the coordinates for them to match: (ovewrite dimentions)
countnegative_mean_sel['longitude'] = lons.values # x_wgs
countnegative_mean_sel['latitude']= lats.values   # y_wgs

In [ ]:
%%time

# to data frame
countnegative_mean_sel_df = countnegative_mean_sel.to_dataframe().reset_index()
#rename:
countnegative_mean_sel_df = countnegative_mean_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})
#rename column: 
countnegative_mean_sel_df = countnegative_mean_sel_df.rename(columns = {'fraction':'belowzero_frac'}) 
# drop month 
countnegative_mean_sel_df = countnegative_mean_sel_df.drop(['t2m','forcount'], axis = 1)

In [ ]:
# merge to the dfclim table on the coordinates:

dfclim = dfclim.merge(countnegative_mean_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()

In [ ]:
# dfclim.columns

## average annual temp < 0

In [ ]:
# pd.Series(np.where(sample.housing.values == 'yes', 1, 0),

dfclim['avgtemp_belowzero'] = np.where(dfclim.mean_annual_T < 0, 1, 0)

In [ ]:
# dfclim.to_csv('2aug_morph_temperature.csv')